---
title: Layers
date: 2023-12-1 
authors:
  - name: Sébastien Boisgérault
    email: Sebastien.Boisgerault@minesparis.psl.eu
    url: https://github.com/boisgera
    affiliations:
      - institution: Mines Paris - PSL University
        department: Institut des Transformation Numériques (ITN)
github: boisgera
license: CC-BY-4.0
open_access: true
---

# Stack of Cards


```{exercise}
1. Create a set of filled rectangles with the following sequence of colors: black, violet, blue, orange green and red. 
2. If necessary, move the rectangles so that they overlap and you can see which one is on top of the other.
3. Which one is at the back of the document? Which one is at the top?  
```

![Layers](images/layers.png)

# The Index
 

```{exercise}
:label: second
1. List the colors of all the shapes in your document, in the order in which they appear. What can you say?
2. In Python, edit your document to make the red rectangle appear *before* every other rectangle. 
Does this change the (relative) depth of this rectangle?
3. List again the colors of all the shapes in your document and the corresponding *index*, a string which is an attribute of the shape.
4. Compare the lexicographic order between these strings and their depth in the document. What can you say?
```

````{note} Lexicographic order

   When Python strings are compared and sorted, by default the lexicographic order is used.

   The lexicographic order generalizes the alphabetical order:

   ```
   >>> "alpaca" < "guanaco" < "lama" < "vicuña"
   True
   ```

   When the first letters are identical, the shorter strings is sorted first:
   ```
   >>> "a" < "alp" < "alpaca"
   True
   ```

   All uppercase letters come before lowercase letters:
   
   ```
   >>> "A" < "Z" < "a" < "z"
   ```

   and therefore 

   ```
   >>> "Vicuña" < "alpaca" < "vicuña"
   True
   ```

   Digits are ordered "naturally":

   ```
   >>> "0" < "1" < "2" < "3" < "4" < "5" < "6" < "7" < "8" < "9"
   True
   ```

   However, beware of the comparison of strings that represent numbers:

   ```
   >>> "2" < "100"
   False
   >>> sorted(["2", "100"])
   ['100', '2']
   ```

   All digits come before letters:

   ```
   >>> "0" < "1" < "9" < "A" < "B" < "Z" < "a" < "b" < "z"
   True
   ```



````

# Fractional Indexing

```{exercise}
1. In the tldraw editor, insert a yellow rectangle into the document and use "Actions/Send backward" repeatedly to put it in a layer between the red and violet rectangles.
2. Save this document and load it in Python.  Did the indices of the old rectangles change? 
3. What is the index of the new rectangle? Is this value consistent with the assumption you made in question 4 of [](#second)?
```

![Yellow rectangle](images/add-yellow.png)

Tldraw uses a technique called **fractional indexing** to generate new indices that fit between the existing ones.
It is explained in details in the [Implementing Fractional Indexing](https://observablehq.com/@dgreensp/implementing-fractional-indexing) Observable (Javascript) notebook.

The core idea of this method is to build a representation of indices as fractions in $\left[0, 1\right[$ which maps the
lexicographic order into the the usual order on $\mathbb{Q}$, 
then to solve the generation of intermediate indices in the fractional space since it's much easier there.

We associate to any string $\mathtt{s}$ using only the 62 characters `"0"`, `"1"`, ... `"9"`, `"A"`, ... `"Z"`, `"a"`, ..., `"z"` as a fraction $\mathcal{F}(\mathtt{s}) \in \left[0, 1\right[$ such that:

$$
\mathcal{F}(\mathtt{""}) = 0
$$

$$
\mathcal{F}(\mathtt{"0"}) = 0, \; \mathcal{F}(\mathtt{"1"}) = \frac{1}{62}, \; \dots
$$

$$
\mathcal{F}(\mathtt{"A"}) = \frac{10}{62}, \; \mathcal{F}(\mathtt{"B"}) = \frac{11}{62}, \; \dots
$$
$$
\mathcal{F}(\mathtt{"a"}) = \frac{36}{62}, \; \mathcal{F}(\mathtt{"b"}) = \frac{37}{62},
\; \mathcal{F}(\mathtt{"z"}) = \frac{61}{62}.
$$

and for any character $\mathtt{c}$ (i.e. string of length 1) and any string $\mathtt{s}$,

$$
\mathcal{F}(\mathtt{c + s}) = \mathcal{F}(\mathtt{c}) + \frac{\mathcal{F}(\mathtt{s})}{62}. 
$$

For example:

$$
\mathcal{F}(\mathtt{"abc"})
= \frac{\mathcal{F}(\mathtt{"a"})}{62} + \frac{\mathcal{F}(\mathtt{"b"})}{62^2} + \frac{\mathcal{F}(\mathtt{"c"})}{62^3}
= \frac{36}{62} + \frac{37}{62^2} + \frac{38}{62^3}
= \frac{35179}{59582}
$$
    

```{exercise}
1. Assume that $\mathcal{F}(\mathtt{s1}) = \mathcal{F}(\mathtt{s2})$. What does this equality tell you about $\mathtt{s1}$ and $\mathtt{s2}$? 
2. Implement $\mathcal{F}$ as `F` using the `fractions` module of the Python standard library.
3. Make sure that all tests in the cell below pass.
```

In [4]:
from fractions import Fraction

ENABLE_TESTS = False # ℹ️ Set to True to test F whenever the cell is executed

def F(string):
    """
    >>> F("") == Fraction(0, 62)
    True
    >>> F("0") == Fraction(0, 62)  # ⚠️ Trailing zero!
    True
    >>> F("1") == Fraction(1, 62)
    True
    >>> F("1000") == Fraction(1, 62)  # ⚠️ Trailing zeros!
    True
    >>> F("9") == Fraction(9, 62)
    True
    >>> F("A") == Fraction(10, 62)
    True
    >>> F("Z") == Fraction(35, 62)
    True
    >>> F("a") == Fraction(36, 62)
    True
    >>> F("z") == Fraction(61, 62)
    True
    
    >>> F("a1") == F("a") + F("1") / 62
    True
    >>> F("a1")
    Fraction(2233, 3844)
    >>> F("a2") == F("a") + F("2") / 62
    True
    >>> F("a2")
    Fraction(1117, 1922)
    >>> F("a3") == F("a") + F("3") / 62
    True
    >>> F("a3")
    Fraction(2235, 3844)

    >>> F("abc") == Fraction(35179, 59582)
    True
    >>> F("aardvark") == Fraction(32218019837031, 54585026396224)
    True
    """
    pass # 🚧 TODO: implement this function

if ENABLE_TESTS: 
    import doctest
    doctest.run_docstring_examples(F, globals())

```{exercise}
1. Implement the inverse of the function $\mathcal{F}$ (restricted to the strings with no trailing zeros) as a function `iF`.
2. Make sure that all tests in the cell below pass.
```

In [5]:
ENABLE_TESTS = False # ℹ️ Set to True to test F whenever the cell is executed

def iF(fraction):
    """
    >>> iF(F("")) == ""
    True
    >>> iF(F("1")) == "1"
    True
    >>> iF(F("A")) == "A"
    True
    >>> iF(F("a")) == "a"
    True
    >>> iF(F("abc")) == "abc"
    True
    >>> iF(F("aardvark")) == "aardvark"
    True
    """
    pass # 🚧 TODO: implement this function

if ENABLE_TESTS: 
    import doctest
    doctest.run_docstring_examples(iF, globals())

```{exercise}
1. Prove that if the strings $\mathtt{s1}$ and $\mathtt{s2}$ have no trailing zeros (e.g. "hello" is ok but not "hell0"),
then $\mathtt{s1} \leq \mathtt{s2}$ (in the lexicographic order) if and only if $\mathcal{F}(\mathtt{s1}) \leq \mathcal{F}(\mathtt{s2})$
(in the usual order on $\mathbb{Q}$).

2. Show that for any valid index (with no trailing zero), the formula

   $$
   \mathtt{index\_3} 
   = 
   \mathcal{F}^{-1}
   \left(
     \frac{
       \mathcal{F}(\mathtt{index\_1}) + \mathcal{F}(\mathtt{index\_2})
     }{2}
   \right)
   $$

   defines a valid index.

3. How are (lexicographically) ordered the strings $\mathtt{index\_1}, \mathtt{index\_2}$ and $\mathtt{index\_3}$?

4. Implement a function `index_between` based on this analysis. Make sure that all the tests in the cell below pass.
```

In [6]:
ENABLE_TESTS = False # ℹ️ Set to True to test F whenever the cell is executed

def index_between(index_1, index_2):
    """
    >>> index_between("1", "2")
    '1V'
    >>> index_between("a", "b")
    'aV'
    >>> index_between("aardvark", "aardwolf")
    'aardwCohV'
    """
    pass # 🚧 TODO: implement this function
    
if ENABLE_TESTS:
    import doctest
    doctest.run_docstring_examples(index_between, globals())

# Application

```{exercise}
1. Go back to your tldraw editor and bring your yellow rectangle to front.
2. Save the corresponding document and load it as a Python object.
3. Use the `index_between` function to patch its depth so that it goes back between the black and violet rectangles.
4. Save the document and reload it into the tldraw editor to check that it worked.
```


![Add yellow on top](images/add-yellow-on-top.png)